<h1 align = 'center'><font size = "10"><b> Long Short-Term Memory Neural Networks <b><font><h1>

## Step 1: Import libraries

In [5]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Bidirectional, LSTM, RepeatVector, Dense, TimeDistributed # for creating layers inside the Neural Network

# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version

# Sklearn
import sklearn
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.preprocessing import MinMaxScaler # for feature scaling

# Visualization
import plotly
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) # print version

pandas: 2.0.3
numpy: 1.25.2
sklearn: 1.2.2
plotly: 5.15.0


## Step 2: Prepare the data. ([Australian weather data from Kaggle](https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package))

Currently I choose subset of columns as we are not in required of whole dataset. Also we can derive couple of new variables like Year-Month, Median Temparture by data manipulation.  

In [7]:
# Set Pandas options to display more columns
pd.options.display.max_columns=150

# Read in the weather data csv - keep only the columns we need
df=pd.read_csv('/weatherAUS.csv', encoding='utf-8', usecols=['Date', 'Location', 'MinTemp', 'MaxTemp'])

# Drop records where target MinTemp=NaN or MaxTemp=NaN
df=df[pd.isnull(df['MinTemp'])==False]
df=df[pd.isnull(df['MaxTemp'])==False]

# Convert dates to year-months
df['Year-Month']= (pd.to_datetime(df['Date'], yearfirst=True)).dt.strftime('%Y-%m')

# Derive median daily temperature (mid point between Daily Max and Daily Min)
df['MedTemp']=df[['MinTemp', 'MaxTemp']].median(axis=1)

# Show a snaphsot of data
df

,Date,Location,MinTemp,MaxTemp,Year-Month,MedTemp
0,2008-12-01,Albury,13.4,22.9,2008-12,18.15
1,2008-12-02,Albury,7.4,25.1,2008-12,16.25
2,2008-12-03,Albury,12.9,25.7,2008-12,19.30
3,2008-12-04,Albury,9.2,28.0,2008-12,18.60
4,2008-12-05,Albury,17.5,32.3,2008-12,24.90
...,...,...,...,...,...,...
145454,2017-06-20,Uluru,3.5,21.8,2017-06,12.65
145455,2017-06-21,Uluru,2.8,23.4,2017-06,13.10
145456,2017-06-22,Uluru,3.6,25.3,2017-06,14.45
145457,2017-06-23,Uluru,5.4,26.9,2017-06,16.15


### Due to daily temparature fluctuations it will be harder for us to predict, so we gonna calculate the monthly averages and transpose the data to have locations as rows and Year-Months as columns.

In [8]:
# Create a copy of an original dataframe
df2=df[['Location', 'Year-Month', 'MedTemp']].copy()

# Calculate monthly average temperature for each location
df2=df2.groupby(['Location', 'Year-Month'], as_index=False).mean()

# Display the new dataframe
df2

,Location,Year-Month,MedTemp
0,Adelaide,2008-07,11.091935
1,Adelaide,2008-08,10.646774
2,Adelaide,2008-09,14.981667
3,Adelaide,2008-10,18.205172
4,Adelaide,2008-11,19.480000
...,...,...,...
4764,Woomera,2017-02,26.889286
4765,Woomera,2017-03,26.238710
4766,Woomera,2017-04,19.251667
4767,Woomera,2017-05,15.108065


In [9]:
# transpose the data and remove the locations and months with missing data.

# Transpose dataframe
df2_pivot=df2.pivot(index=['Location'], columns='Year-Month')['MedTemp']

# Remove locations with lots of missing (NaN) data
df2_pivot=df2_pivot.drop(['Dartmoor', 'Katherine', 'Melbourne', 'Nhil', 'Uluru'], axis=0)

# Remove months with lots of missing (NaN) data
df2_pivot=df2_pivot.drop(['2007-11', '2007-12', '2008-01', '2008-02', '2008-03', '2008-04', '2008-05', '2008-06', '2008-07', '2008-08', '2008-09', '2008-10', '2008-11', '2008-12', '2017-01', '2017-02', '2017-03', '2017-04', '2017-05', '2017-06'], axis=1)

# Display the new dataframe
df2_pivot

Year-Month,2009-01,2009-02,2009-03,2009-04,2009-05,2009-06,2009-07,2009-08,2009-09,2009-10,2009-11,2009-12,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,2010-10,2010-11,2010-12,2011-01,2011-02,2011-03,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2013-01,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12
Location,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Adelaide,24.967742,25.057143,20.685484,17.908333,14.324194,12.540000,12.388710,13.832258,15.026667,16.708065,24.428333,22.208065,24.737097,25.203571,21.956452,19.165000,15.106452,11.660000,11.125806,11.524194,13.071667,16.535484,19.228333,21.364516,24.467742,23.846429,19.911290,13.970968,12.521667,11.750000,13.620968,15.510000,17.570968,20.723333,22.129032,25.146774,22.686207,20.612903,18.388333,13.767742,11.573333,11.577419,11.769355,14.823333,16.943548,21.375000,23.545161,23.259677,18.551667,16.445161,12.521667,12.674194,13.162903,17.485000,16.816129,19.110000,22.043548,25.524194,23.801786,21.274194,18.163333,16.420968,12.838333,11.750000,12.087097,15.848333,19.058065,20.636667,21.188710,23.103226,25.373214,20.003226,15.785000,14.235484,12.151667,10.708065,11.988710,14.090000,20.554839,20.960345,25.285484,24.662903,22.829310,22.730645,18.708333,16.091935,12.466667,11.696774,12.696774,13.241667,15.790323,18.413333,22.025806
Albany,19.861667,20.150000,18.433871,17.461667,15.417742,13.321667,12.920968,12.988710,12.568333,15.329032,17.300000,17.864516,19.703226,20.308929,19.859677,16.978333,15.064516,13.283333,12.435484,12.838710,13.653333,15.380645,18.058333,17.925806,20.458065,20.287500,19.893548,16.233871,14.636667,12.320968,12.985484,13.843333,15.561290,17.508333,19.045161,20.425000,20.067241,19.993548,18.801786,15.741935,14.075000,13.191935,13.324194,14.241667,16.791935,17.108621,20.798333,19.081034,18.883333,15.575806,13.978333,13.253226,14.317742,14.621667,15.345161,17.705000,18.000000,20.301667,19.577778,19.277419,17.736538,16.101613,14.646667,13.205000,14.248387,15.226667,16.030645,17.093333,17.716667,20.250000,20.828571,19.143548,17.382143,14.706897,14.860000,13.290000,13.403333,14.620000,16.683333,17.585000,18.233333,20.189286,20.168966,19.660345,17.583333,14.827419,13.235000,12.891667,12.590476,12.466667,14.232759,16.489286,17.482000
Albury,25.485484,25.439286,20.535484,15.410000,11.716129,9.206667,8.201613,9.932258,11.951786,14.595161,22.803333,22.641935,25.780645,25.074000,20.775926,16.338333,11.580645,8.455000,7.874194,8.320000,11.016667,15.000000,19.258333,20.793548,24.666129,22.621429,19.366129,10.111290,8.643333,8.051613,10.150000,11.925000,15.233871,20.068333,20.800000,23.882258,22.163793,18.769355,15.668333,10.085484,8.018333,8.316129,8.567742,11.660000,14.000000,19.743333,25.629032,21.091935,15.936667,10.883333,8.833333,9.808065,10.379032,13.591667,14.295000,18.108333,22.727419,25.425806,24.946429,21.233871,16.341667,12.630645,9.943333,8.514516,8.820968,12.153704,16.606452,20.373333,22.393548,23.475806,25.000000,19.832258,15.983333,11.388710,7.848333,7.638710,8.662903,11.691667,18.975806,20.603333,23.501613,24.456452,24.329310,23.290323,17.580000,13.069355,9.500000,9.517742,9.625806,12.213333,13.617742,18.170000,22.677419
AliceSprings,29.240323,28.642857,27.470968,21.181667,14.437097,13.580000,12.933871,19.232258,22.040000,22.020968,27.345000,27.887097,28.346774,27.805357,23.753226,21.431667,15.480645,11.110000,12.153226,12.800000,16.236667,19.245161,22.210000,26.066129,29.827419,27.510714,23.345161,13.266129,10.980000,12.001613,15.219355,18.403333,23.333871,25.146667

### Impute the missing values
If you notice we are missing few months of data like 2012-12, 2013-02 etc. Therefore, we impute values for the missing months by taking an average of the preceding and subsequent month.



In [10]:
# Add missing months 2011-04, 2011-04, 2011-04 and impute data
df2_pivot['2011-04']=(df2_pivot['2011-03']+df2_pivot['2011-05'])/2
df2_pivot['2012-12']=(df2_pivot['2012-11']+df2_pivot['2013-01'])/2
df2_pivot['2013-02']=(df2_pivot['2013-01']+df2_pivot['2013-03'])/2

# Sort columns so Year-Months are in the correct order
df2_pivot=df2_pivot.reindex(sorted(df2_pivot.columns), axis=1)

# Display updated dataframe
df2_pivot

Year-Month,2009-01,2009-02,2009-03,2009-04,2009-05,2009-06,2009-07,2009-08,2009-09,2009-10,2009-11,2009-12,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,2010-10,2010-11,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12
Location,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Adelaide,24.967742,25.057143,20.685484,17.908333,14.324194,12.540000,12.388710,13.832258,15.026667,16.708065,24.428333,22.208065,24.737097,25.203571,21.956452,19.165000,15.106452,11.660000,11.125806,11.524194,13.071667,16.535484,19.228333,21.364516,24.467742,23.846429,19.911290,16.941129,13.970968,12.521667,11.750000,13.620968,15.510000,17.570968,20.723333,22.129032,25.146774,22.686207,20.612903,18.388333,13.767742,11.573333,11.577419,11.769355,14.823333,16.943548,21.375000,22.460081,23.545161,23.402419,23.259677,18.551667,16.445161,12.521667,12.674194,13.162903,17.485000,16.816129,19.110000,22.043548,25.524194,23.801786,21.274194,18.163333,16.420968,12.838333,11.750000,12.087097,15.848333,19.058065,20.636667,21.188710,23.103226,25.373214,20.003226,15.785000,14.235484,12.151667,10.708065,11.988710,14.090000,20.554839,20.960345,25.285484,24.662903,22.829310,22.730645,18.708333,16.091935,12.466667,11.696774,12.696774,13.241667,15.790323,18.413333,22.025806
Albany,19.861667,20.150000,18.433871,17.461667,15.417742,13.321667,12.920968,12.988710,12.568333,15.329032,17.300000,17.864516,19.703226,20.308929,19.859677,16.978333,15.064516,13.283333,12.435484,12.838710,13.653333,15.380645,18.058333,17.925806,20.458065,20.287500,19.893548,18.063710,16.233871,14.636667,12.320968,12.985484,13.843333,15.561290,17.508333,19.045161,20.425000,20.067241,19.993548,18.801786,15.741935,14.075000,13.191935,13.324194,14.241667,16.791935,17.108621,18.953477,20.798333,19.939684,19.081034,18.883333,15.575806,13.978333,13.253226,14.317742,14.621667,15.345161,17.705000,18.000000,20.301667,19.577778,19.277419,17.736538,16.101613,14.646667,13.205000,14.248387,15.226667,16.030645,17.093333,17.716667,20.250000,20.828571,19.143548,17.382143,14.706897,14.860000,13.290000,13.403333,14.620000,16.683333,17.585000,18.233333,20.189286,20.168966,19.660345,17.583333,14.827419,13.235000,12.891667,12.590476,12.466667,14.232759,16.489286,17.482000
Albury,25.485484,25.439286,20.535484,15.410000,11.716129,9.206667,8.201613,9.932258,11.951786,14.595161,22.803333,22.641935,25.780645,25.074000,20.775926,16.338333,11.580645,8.455000,7.874194,8.320000,11.016667,15.000000,19.258333,20.793548,24.666129,22.621429,19.366129,14.738710,10.111290,8.643333,8.051613,10.150000,11.925000,15.233871,20.068333,20.800000,23.882258,22.163793,18.769355,15.668333,10.085484,8.018333,8.316129,8.567742,11.660000,14.000000,19.743333,22.686183,25.629032,23.360484,21.091935,15.936667,10.883333,8.833333,9.808065,10.379032,13.591667,14.295000,18.108333,22.727419,25.425806,24.946429,21.233871,16.341667,12.630645,9.943333,8.514516,8.820968,12.153704,16.606452,20.373333,22.393548,23.475806,25.000000,19.832258,15.983333,11.388710,7.848333,7.638710,8.662903,11.691667,18.975806,20.603333,23.501613,24.456452,24.329310,23.290323,17.580000,13.069355,9.500000,9.517742,9.625806,12.213333,13.617742,18.170000,22.677419
AliceSprings,29.240323,28.642857,27.470968,21.181667,14.437097,13.580000,12.933871,19.232258,22.040000,22.020968,27.345000,27.887097,28.346774,27.805357,23.753226,21.431667,15.480645,11.110000,12.153226,12.800000,16.236667,19.245161,22

### Plot the data for each city.

In [11]:
# Plot average monthly temperature derived from daily medians for each location
fig = go.Figure()
for location in df2_pivot.index:
    fig.add_trace(go.Scatter(x=df2_pivot.loc[location, :].index,
                             y=df2_pivot.loc[location, :].values,
                             mode='lines',
                             name=location,
                             opacity=0.8,
                             line=dict(width=1)
                            ))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'), showlegend=True)

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey',
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey',
                 showline=True, linewidth=1, linecolor='black',
                 title='Date'
                )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey',
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey',
                 showline=True, linewidth=1, linecolor='black',
                 title='Degrees Celsius'
                )

# Set figure title
fig.update_layout(title=dict(text="Average Monthly Temperatures", font=dict(color='black')))

fig.show()

## Step 3: Train and evaluate the LSTM model
- We want to train a model for an individual location
- We want to use "X" number of months to predict the next "X" number months
- We want to create multiple sequences for training data where each one starts at a different time (month)
- We want to divide data into train and test samples where test sample will be used for out of time validation

Setup a function to prepare data for LSTM

In [12]:
def shaping(datain, timestep):

    # Convert input dataframe to array and flatten
    arr=datain.to_numpy().flatten()

    cnt=0
    for mth in range(0, len(datain.columns)-(2*timestep)+1): # Define range
        cnt=cnt+1 # Gives us the number of samples. Later used to reshape the data
        X_start=mth # Start month for inputs of each sample
        X_end=mth+timestep # End month for inputs of each sample
        Y_start=mth+timestep # Start month for targets of each sample. Note, start is inclusive and end is exclusive, that's why X_end and Y_start is the same number
        Y_end=mth+2*timestep # End month for targets of each sample.

        # Assemble input and target arrays containing all samples
        if mth==0:
            X_comb=arr[X_start:X_end]
            Y_comb=arr[Y_start:Y_end]
        else:
            X_comb=np.append(X_comb, arr[X_start:X_end])
            Y_comb=np.append(Y_comb, arr[Y_start:Y_end])

    # Reshape input and target arrays arrays
    X_out=np.reshape(X_comb, (cnt, timestep, 1))
    Y_out=np.reshape(Y_comb, (cnt, timestep, 1))
    return X_out, Y_out

In [13]:
### My individual location pick is Albury and my X number is 18 to predict next 18 months. Remember if you are having the smae lengths of input and output sequences you choose to set return_sequences = True in the encoder layer and remove the repeat vector
### A repeat vector is necessary when we want to have sequences of different lengths
### we added a Bidirectional wrapper to LSTM layers. It allows us to train the model in both directions, which sometimes produces better results. However, its use is optional.
### Also, we need to use a Time Distributed wrapper in the output layer to predict outputs for each timestep individually.
### All these you can find in the step 3

##### Step 1 - Specify parameters
timestep=18
location='Albury'


##### Step 2 - Prepare data

# Split data into train and test dataframes
df_train=df2_pivot.iloc[:, 0:-2*timestep].copy()
df_test=df2_pivot.iloc[:, -2*timestep:].copy()

# Select one location
dfloc_train = df_train[df_train.index==location].copy()
dfloc_test = df_test[df_test.index==location].copy()

# Use previously defined shaping function to reshape the data for LSTM
X_train, Y_train = shaping(datain=dfloc_train, timestep=timestep)
X_test, Y_test = shaping(datain=dfloc_test, timestep=timestep)


##### Step 3 - Specify the structure of a Neural Network
model = Sequential(name="LSTM-Model") # Model
model.add(Input(shape=(X_train.shape[1],X_train.shape[2]), name='Input-Layer')) # Input Layer - need to speicfy the shape of inputs
model.add(Bidirectional(LSTM(units=32, activation='tanh', recurrent_activation='sigmoid', stateful=False), name='Hidden-LSTM-Encoder-Layer')) # Encoder Layer
model.add(RepeatVector(Y_train.shape[1], name='Repeat-Vector-Layer')) # Repeat Vector
model.add(Bidirectional(LSTM(units=32, activation='tanh', recurrent_activation='sigmoid', stateful=False, return_sequences=True), name='Hidden-LSTM-Decoder-Layer')) # Decoder Layer
model.add(TimeDistributed(Dense(units=1, activation='linear'), name='Output-Layer')) # Output Layer, Linear(x) = x


##### Step 4 - Compile the model
model.compile(optimizer='adam', # default='rmsprop', an algorithm to be used in backpropagation
              loss='mean_squared_error', # Loss function to be optimized. A string (name of loss function), or a tf.keras.losses.Loss instance.
              metrics=['MeanSquaredError', 'MeanAbsoluteError'], # List of metrics to be evaluated by the model during training and testing. Each of this can be a string (name of a built-in function), function or a tf.keras.metrics.Metric instance.
              loss_weights=None, # default=None, Optional list or dictionary specifying scalar coefficients (Python floats) to weight the loss contributions of different model outputs.
              weighted_metrics=None, # default=None, List of metrics to be evaluated and weighted by sample_weight or class_weight during training and testing.
              run_eagerly=None, # Defaults to False. If True, this Model's logic will not be wrapped in a tf.function. Recommended to leave this as None unless your Model cannot be run inside a tf.function.
              steps_per_execution=None # Defaults to 1. The number of batches to run during each tf.function call. Running multiple batches inside a single tf.function call can greatly improve performance on TPUs or small models with a large Python overhead.
             )


##### Step 5 - Fit the model on the dataset
history = model.fit(X_train, # input data
                    Y_train, # target data
                    batch_size=1, # Number of samples per gradient update. If unspecified, batch_size will default to 32.
                    epochs=1000, # default=1, Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided
                    verbose=0, # default='auto', ('auto', 0, 1, or 2). Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. 'auto' defaults to 1 for most cases, but 2 when used with ParameterServerStrategy.
                    callbacks=None, # default=None, list of callbacks to apply during training. See tf.keras.callbacks
                    validation_split=0.2, # default=0.0, Fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch.
                    #validation_data=(X_test, y_test), # default=None, Data on which to evaluate the loss and any model metrics at the end of each epoch.
                    shuffle=True, # default=True, Boolean (whether to shuffle the training data before each epoch) or str (for 'batch').
                    class_weight=None, # default=None, Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only). This can be useful to tell the model to "pay more attention" to samples from an under-represented class.
                    sample_weight=None, # default=None, Optional Numpy array of weights for the training samples, used for weighting the loss function (during training only).
                    initial_epoch=0, # Integer, default=0, Epoch at which to start training (useful for resuming a previous training run).
                    steps_per_epoch=None, # Integer or None, default=None, Total number of steps (batches of samples) before declaring one epoch finished and starting the next epoch. When training with input tensors such as TensorFlow data tensors, the default None is equal to the number of samples in your dataset divided by the batch size, or 1 if that cannot be determined.
                    validation_steps=None, # Only relevant if validation_data is provided and is a tf.data dataset. Total number of steps (batches of samples) to draw before stopping when performing validation at the end of every epoch.
                    validation_batch_size=None, # Integer or None, default=None, Number of samples per validation batch. If unspecified, will default to batch_size.
                    validation_freq=100, # default=1, Only relevant if validation data is provided. If an integer, specifies how many training epochs to run before a new validation run is performed, e.g. validation_freq=2 runs validation every 2 epochs.
                    max_queue_size=10, # default=10, Used for generator or keras.utils.Sequence input only. Maximum size for the generator queue. If unspecified, max_queue_size will default to 10.
                    workers=1, # default=1, Used for generator or keras.utils.Sequence input only. Maximum number of processes to spin up when using process-based threading. If unspecified, workers will default to 1.
                    use_multiprocessing=True, # default=False, Used for generator or keras.utils.Sequence input only. If True, use process-based threading. If unspecified, use_multiprocessing will default to False.
                   )


##### Step 6 - Use model to make predictions
# Predict results on training data
pred_train = model.predict(X_train)
# Predict esults on test data
pred_test = model.predict(X_test)


##### Step 7 - Print Performance Summary
print("")
print('-------------------- Model Summary --------------------')
model.summary() # print model summary
print("")
print('-------------------- Weights and Biases --------------------')
print("Too many parameters to print but you can use the code provided if needed")
print("")
#for layer in model.layers:
#    print(layer.name)
#    for item in layer.get_weights():
#        print("  ", item)
#print("")

# Print the last value in the evaluation metrics contained within history file
print('-------------------- Evaluation on Training Data --------------------')
for item in history.history:
    print("Final", item, ":", history.history[item][-1])
print("")

# Evaluate the model on the test data using "evaluate"
print('-------------------- Evaluation on Test Data --------------------')
results = model.evaluate(X_test, Y_test)
print("")

1/1 [==============================] - 0s 26ms/step

-------------------- Model Summary --------------------
Model: "LSTM-Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Hidden-LSTM-Encoder-Layer   (None, 64)                8704      
 (Bidirectional)                                                 
                                                                 
 Repeat-Vector-Layer (Repea  (None, 18, 64)            0         
 tVector)                                                        
                                                                 
 Hidden-LSTM-Decoder-Layer   (None, 18, 64)            24832     
 (Bidirectional)                                                 
                                                                 
 Output-Layer (TimeDistribu  (None, 18, 1)             65        
 ted)                                                            
             

## Step 4: Results and visualization.

In [14]:
# Plot average monthly temperatures (actual and predicted) for test (out of time) data
fig = go.Figure()

# Trace for actual temperatures
fig.add_trace(go.Scatter(x=np.array(dfloc_test.columns),
                         y=np.array(dfloc_test.values).flatten(),
                         mode='lines',
                         name='Average Monthly Temperatures - Actual (Test)',
                         opacity=0.8,
                         line=dict(color='black', width=1)
                        ))

# Trace for predicted temperatures
fig.add_trace(go.Scatter(x=np.array(dfloc_test.columns[-timestep:]),
                         y=pred_test.flatten(),
                         mode='lines',
                         name='Average Monthly Temperatures - Predicted (Test)',
                         opacity=0.8,
                         line=dict(color='red', width=1)
                        ))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey',
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey',
                 showline=True, linewidth=1, linecolor='black',
                 title='Year-Month'
                )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey',
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey',
                 showline=True, linewidth=1, linecolor='black',
                 title='Degrees Celsius'
                )

# Set figure title
fig.update_layout(title=dict(text="Average Monthly Temperatures", font=dict(color='black')),
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
                 )
fig.show()

<p align='center'><font size = ""> -- End of program -- <font><p>
<p align='center'><font size = "10"><b> Happy coding! 😀 <b><font><p>